# Generate tiled region run file

## This notebook is an example: create a copy before running it or you will get merge conflicts!

**NOTE**: before running this on a new MIBI, run `update_coregistraion_params.ipynb` first. This will run co-registration so your FOVs display correctly on the slide.

This notebook automatically generates FOVs across multiple tiled regions on a sample and allows you to remove unneeded FOVs created during the process.

In [ ]:
import sys
sys.path.append('../')

In [ ]:
import json
import os
from skimage.io import imread

from toffy import tiling_utils

# suppress mpl deprecation
import warnings
from matplotlib.cbook import mplDeprecation
warnings.filterwarnings("ignore", category=mplDeprecation)

### Define paths to the JSON data

Define the following parameters to set your home directory:

* `base_dir`: the root folder, `json_tiling_data` should be placed here
* `json_tiling_dir`: the directory containing the information to read and write the FOV info
* `slide_path`: the path to the slide image of the sample

Define the following input and output paths:

* `region_corners_path`: the JSON file containing the FOVs defining the top-left corner of each region
* `tiled_region_fovs_path`: where to store JSON defining the FOVs for each tiled region
* `moly_path`: the path to the Moly point, needed if you want to insert between FOVs
* `tiled_region_path`: the path to save the final set FOVs for each tiled region

In [ ]:
# define the tiling directory
base_dir = "data_dir"
json_tiling_dir = os.path.join(base_dir, "json_tiling_data")

# define the prefix of each file
tma_prefix = "example_tiled_region"

# define the slide path
slide_path = os.path.join(json_tiling_dir, "%s_slide.png" % tma_prefix)

In [ ]:
# define the input and output files
region_corners_path = os.path.join(json_tiling_dir, 'region_corners.json')
tiled_region_fovs_path = os.path.splitext(region_corners_path)[0] + '_tiled_fovs.json'
moly_path = os.path.join(json_tiling_dir, '%s_moly_point.json' % tma_prefix)
tiled_region_path = os.path.join(json_tiling_dir, "sample_tiled_region_mapping.json")

### Set tiling parameters

The following parameters need to be defined for each region:

* The FOV defining the top-left corner with its centroid (taken from `region_corners_path`, each entry indicates a different region)
* Number of FOVs along the row and column axes (user prompt)
* row- and column-axis step size (taken from the `fovSizeMicrons` field of each corner FOV in `region_corners_path`, same for row and column)
* Whether to randomize the order of the FOVs or not (user prompt)
* Whether to insert Moly points between regions (user prompt)
* Whether to insert Moly points between FOVs at a specified interval (user prompt). Refer to this link for how Moly point insertion works: https://ark-analysis.readthedocs.io/en/latest/_markdown/ark.mibi.html#ark.mibi.tiling_utils.tiled_region_generate_fov_list

In [ ]:
tiling_params = tiling_utils.set_tiled_region_params(region_corners_path)

### Create the FOVs for each region

FOV names correspond to their row and column position on the grid of FOVs and are 1-indexed. For example, `R1C1` means the FOV is in row 1 and column 1, `R2C1` would be the next FOV down, and `R1C2` would be the next FOV to the right. The top left FOV is `R1C1`, the bottom right FOV would be `R4C5` for a tiled region of 4x5 dimensions. 

In [ ]:
# generate the FOVs in each region
tiled_region_fovs = tiling_utils.generate_tiled_region_fov_list(
    tiling_params,
    moly_path
)

### Interactive FOV deselection

The grid defining an ROI may cover more area than you wish to tile. This GUI allows you to interactively delete FOVs from `tiled_region_fovs` prior to saving:

* Click on a FOV to propose it for removal. The border of proposed FOVs will be highlighted. Click again to unpropose the FOV.
* Once you've selected all the FOVs you want to remove, click `Delete selected FOVs` to remove them. <b>This action cannot be undone!</b>
* You can always remove more FOVs even after saving. Just remember to re-save.
* After you're done removing all the FOVs, click `Save mapping` to save your truncated set of FOVs to `tiled_region_path`

In [ ]:
# read the slide image in
slide_data = imread(slide_path)

In [ ]:
%matplotlib widget
_ = tiling_utils.tiled_region_interactive_remap(
    tiled_region_fovs,
    tiling_params,
    slide_data,
    tiled_region_path,
    figsize=(7, 7)
)